In [0]:
# !git config --global user.email "me@example.com"
# !git config --global user.name "nervuzz"
# !git commit -m 'Day 5'
# !git pull
# !git add .
# !git status
# !git push

In [1]:
cd "drive/My Drive/Colab Notebooks/data_workshop_tsr/"

/content/drive/My Drive/Colab Notebooks/data_workshop_tsr


In [0]:
# !pip install hyperopt

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

# train.keys(), test.keys()

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
# We have to create a matrix with n columns where n is number of unique labels
# Each row is filled with 0es and only one column has value 1
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
  )

  logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(
    X_train, y_train,
    batch_size=params_fit.get('batch_size', 128),
    epochs=params_fit.get('epochs', 5),
    verbose=params_fit.get('verbose', 1),
    validation_data=params_fit.get('validation_data', (X_train, y_train)),
    callbacks=[tensorboard_callback]
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(in_shp, num_cls, params={}):
  return Sequential([
    Conv2D(
        filters=params.get('filters', 32),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
        input_shape=in_shp,
    ),
    Conv2D(
        filters=params.get('filters', 32),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
        input_shape=in_shp,
    ),
    Conv2D(
        filters=params.get('filters', 32),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
        padding='same',
    ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(
        filters=params.get('filters', 64),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
        padding='same',
    ),
    Conv2D(
        filters=params.get('filters', 64),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
    ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(
        filters=params.get('filters', 64),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
        padding='same',
    ),
    Conv2D(
        filters=params.get('filters', 64),
        kernel_size=params.get('kernel_size', (3, 3)),
        activation=params.get('activation', 'relu'),
    ),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [29]:
train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 14s 52ms/step - loss: 2.4898 - accuracy: 0.2903 - val_loss: 1.1280 - val_accuracy: 0.5963
Epoch 2/5
272/272 [==============================] - 14s 51ms/step - loss: 0.8679 - accuracy: 0.7064 - val_loss: 0.2817 - val_accuracy: 0.9116
Epoch 3/5
272/272 [==============================] - 14s 51ms/step - loss: 0.3691 - accuracy: 0.8822 - val_loss: 0.0982 - val_accuracy: 0.9719
Epoch 4/5
272/272 [==============================] - 14s 51ms/step - loss: 0.2462 - accuracy: 0.9236 - val_loss: 0.0512 - val_accuracy: 0.9859
Epoch 5/5
272/272 [==============================] - 14s 51ms/step - loss: 0.1728 - accuracy: 0.9469 - val_loss: 0.0441 - val_accuracy: 0.9878


0.9675736961451247

**New function for Hyperopt**

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
    X_train, y_train,
    batch_size=int(params.get('batch_size', 128)),
    epochs=5,
    verbose=0
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(f'{params}, accuracy={accuracy}')

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [37]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(func_obj,
            space,
            tpe.suggest,
            30,
            Trials()
          )

  0%|          | 0/30 [00:18<?, ?it/s, best loss: ?]


KeyboardInterrupt: ignored

In [38]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/nervuzz/data_workshop_tsr
   bd87246..615978d  master     -> origin/master
Updating bd87246..615978d
Fast-forward
 README.md | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
